In [1]:
import requests
import pandas as pd
import datetime as dt


api_key = "26f63d53-90e3-459c-b7ba-1acb658f2f55"

DMI_URL = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items'
r = requests.get(DMI_URL, params={'api-key': api_key}) # Issues a HTTP GET request
json = r.json()  # Extract JSON data
df = pd.json_normalize(json['features'])  # Convert JSON object to a Pandas DataFrame
df.head()  # Print the first five rows of the DataFrame
df['time'] = pd.to_datetime(df['properties.observed'])


/Users/rasmusmonsted/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
parameterIds = df['properties.parameterId'].unique()  # Generate a list of unique parameter ids
print(parameterIds)  # Print all unique parameter ids

['temp_dew' 'humidity' 'cloud_cover' 'temp_max_past12h' 'pressure_at_sea'
 'wind_speed' 'temp_soil_max_past1h' 'weather' 'wind_dir' 'temp_dry'
 'wind_dir_past1h' 'precip_past1min' 'precip_dur_past10min' 'temp_grass'
 'leav_hum_dur_past10min' 'temp_min_past1h' 'precip_past1h' 'pressure'
 'radia_glob' 'wind_speed_past1h' 'humidity_past1h' 'sun_last10min_glob'
 'precip_past10min' 'visibility' 'visib_mean_last10min'
 'leav_hum_dur_past1h' 'temp_soil' 'temp_soil_min_past1h' 'wind_min'
 'temp_grass_min_past1h' 'wind_min_past1h' 'cloud_height'
 'temp_min_past12h' 'wind_max_per10min_past1h' 'temp_max_past1h'
 'temp_soil_mean_past1h' 'wind_max' 'radia_glob_past1h'
 'temp_grass_mean_past1h' 'precip_dur_past1h' 'wind_gust_always_past1h'
 'sun_last1h_glob' 'temp_mean_past1h' 'temp_grass_max_past1h'
 'snow_depth_man']


In [ ]:
# Specify the desired start and end time
startTime = pd.Timestamp(2022, 1, 1)
endTime = pd.Timestamp(2022, 1, 15)

# Specify one or more station IDs or all_stations
stationIds = [
    '04203', '04208', '04214', '04220', '04228', '04242', '04250',
    '04253', '04266', '04271', '04272', '04285', '04301', '04312',
    '04313', '04320', '04330', '04339', '04351', '04360', '04373',
    '04382', '04390', '05005', '05009', '05015', '05031', '05035',
    '05042', '05065', '05070', '05075', '05081', '05085', '05089',
    '05095', '05105', '05109', '05135', '05140', '05150', '05160',
    '05165', '05169', '05185', '05199', '05202', '05205', '05220',
    '05225', '05269', '05272', '05276', '05277', '05290', '05296',
    '05300', '05305', '05320', '05329', '05343', '05345', '05350',
    '05355', '05365', '05375', '05381', '05395', '05400', '05406',
    '05408', '05435', '05440', '05450', '05455', '05469', '05499',
    '05505', '05510', '05529', '05537', '05545', '05575', '05735',
    '05880', '05889', '05935', '05945', '05970', '05986', '05994',
    '06019', '06031', '06032', '06041', '06049', '06051', '06052',
    '06056', '06058', '06065', '06068', '06072', '06073', '06074',
    '06079', '06081', '06082', '06088', '06093', '06096', '06102',
    '06116', '06119', '06123', '06124', '06126', '06132', '06135',
    '06136', '06138', '06141', '06147', '06149', '06151', '06154',
    '06156', '06159', '06168', '06169', '06174', '06181', '06183',
    '06184', '06186', '06187', '06188', '06193', '06197', '20000',
    '20030', '20055', '20085', '20228', '20279', '20315', '20375',
    '20400', '20552', '20561', '20600', '20670', '21020', '21080',
    '21100', '21120', '21160', '21208', '21368', '21430', '22020',
    '22080', '22162', '22189', '22232', '22410', '23100', '23133',
    '23160', '23327', '23360', '24043', '24102', '24142', '24171',
    '24380', '24430', '24490', '25045', '25161', '25270', '25339',
    '26210', '26340', '26358', '26450', '27008', '27082', '28032',
    '28110', '28240', '28280', '28385', '28552', '28590', '29020',
    '29194', '29243', '29330', '29440', '30075', '30187', '30215',
    '30414', '31040', '31185', '31199', '31259', '31350', '31400',
    '31509', '31570', '32110', '32175', '34270', '34320', '34339'
]
# Specify one or more parameter IDs or all_parameters
parameterIds = [
    # Humdity
    'humidity', 'humidity_past1h',
    # Precipitation
    'precip_past10min', 'precip_past1h', 'precip_past24h',   
    # Temperature
    'temp_dew', 'temp_dry', 'temp_max_past12h', 'temp_max_past1h',
    'temp_mean_past1h', 'temp_min_past12h', 'temp_min_past1h',
]

# Derive datetime specifier string
datetime_str = startTime.tz_localize('UTC').isoformat() + '/' + endTime.tz_localize('UTC').isoformat()

dfs = []
for station in stationIds:
    for parameter in parameterIds:
        # Specify query parameters
        params = {
            'api-key' : api_key,
            'datetime' : datetime_str,
            'stationIds' : station,
            'parameterId' : parameter,
            'limit' : '300000',  # max limit
        }

        # Submit GET request with url and parameters
        r = requests.get(DMI_URL, params=params)
        # Extract JSON object
        json = r.json() # Extract JSON object
        # Convert JSON object to a MultiIndex DataFrame and add to list
        dfi = pd.json_normalize(json['features'])
        if dfi.empty is False:
            dfi['time'] = pd.to_datetime(dfi['properties.observed'])
            # Drop other columns
            dfi = dfi[['time', 'properties.value', 'properties.stationIds', 'properties.parameterId']]
            # Rename columns, e.g., 'properties.stationIds' becomes 'stationIds'
            dfi.columns = [c.replace('properties.', '') for c in dfi.columns]
            # Drop identical rows (considers both value and time stamp)
            dfi = dfi[~dfi.duplicated()]
            dfi = dfi.set_index(['parameterId', 'stationIds', 'time'])
            dfi = dfi['value'].unstack(['stationIds','parameterId'])
            dfs.append(dfi)

    # create a log for each station id
    print(f"Station {station}/{len(stationIds)} has been processed")        

df = pd.concat(dfs, axis='columns').sort_index()
df.head()



In [ ]:
import requests
import json
import pandas as pd
import datetime
import concurrent.futures
from typing import Dict, List, Optional, Union, Tuple

# Base URL for DMI Meteorological Observation API
BASE_URL = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"

def fetch_single_request(api_key: str, 
                        stationId: str, 
                        parameterIds: str, 
                        datetime_str: str,
                        limit: int = 300000) -> Tuple[str, str, pd.DataFrame]:
    """
    Fetch data for a single station-parameter combination.
    
    Args:
        api_key: Your DMI API key
        stationId: Station ID
        parameterIds: Parameter ID
        datetime_str: Formatted datetime range string
        limit: Maximum number of records to return
        
    Returns:
        Tuple containing stationId, parameterIds, and resulting DataFrame
    """
    headers = {"X-API-Key": api_key}
    
    params = {
        'datetime': datetime_str,
        'stationIds': stationId,
        'parameterId': parameterIds,
        'limit': str(limit),
    }
    
    try:
        response = requests.get(BASE_URL, headers=headers, params=params)
        
        if response.status_code == 200:
            json_data = response.json()
            df = pd.json_normalize(json_data.get('features', []))
            
            if not df.empty:
                df['time'] = pd.to_datetime(df['properties.observed'])
                df = df[['time', 'properties.value', 'properties.stationIds', 'properties.parameterId']]
                df.columns = [c.replace('properties.', '') for c in df.columns]
                df = df[~df.duplicated()]
                
                return stationId, parameterIds, df
        
        print(f"No data or error for station {stationId}, parameter {parameterIds}")
        return stationId, parameterIds, pd.DataFrame()
        
    except Exception as e:
        print(f"Error for station {stationId}, parameter {parameterIds}: {str(e)}")
        return stationId, parameterIds, pd.DataFrame()

def get_all_data_concurrent(api_key: str,
                           stationIds: List[str],
                           parameterIds: List[str],
                           startTime: pd.Timestamp,
                           endTime: pd.Timestamp,
                           max_workers: int = 10,
                           limit: int = 300000) -> pd.DataFrame:
    """
    Fetch meteorological data for multiple stations and parameters concurrently.
    
    Args:
        api_key: Your DMI API key
        stationIds: List of station IDs
        parameterIds: List of parameter IDs
        startTime: Start timestamp
        endTime: End timestamp
        max_workers: Maximum number of concurrent requests
        limit: Maximum records per request
        
    Returns:
        DataFrame with meteorological data
    """
    # Convert timestamps to ISO format with UTC timezone
    datetime_str = startTime.tz_localize('UTC').isoformat() + '/' + endTime.tz_localize('UTC').isoformat()
    
    all_dfs = []
    total_tasks = len(stationIds) * len(parameterIds)
    completed = 0
    
    # Use ThreadPoolExecutor for concurrent requests
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Create a list of futures
        futures = []
        for station in stationIds:
            for parameter in parameterIds:
                futures.append(
                    executor.submit(
                        fetch_single_request, 
                        api_key, 
                        station, 
                        parameter, 
                        datetime_str, 
                        limit
                    )
                )
        
        # Process completed futures
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            try:
                stationId, parameterId, df = future.result()
                if not df.empty:
                    print(f"Completed {completed}/{total_tasks}: Station {stationId}, Parameter {parameterIds}")
                    all_dfs.append(df)
                else:
                    print(f"No data for {completed}/{total_tasks}: Station {stationId}, Parameter {parameterIds}")
            except Exception as e:
                print(f"Error processing task {completed}/{total_tasks}: {str(e)}")
    
    # Combine all dataframes
    if all_dfs:
        # Concatenate all dataframes
        combined_df = pd.concat(all_dfs, ignore_index=True)
        
        # Set multi-index and pivot the DataFrame
        # First, convert value column to numeric to ensure proper aggregation
        combined_df['value'] = pd.to_numeric(combined_df['value'], errors='coerce')
        
        # Create the multi-index and unstack
        pivoted_df = combined_df.pivot_table(
            index='time', 
            columns=['parameterId', 'stationIds'], 
            values='value',
            aggfunc='mean'  # In case of duplicates
        )
        
        return pivoted_df
    else:
        return pd.DataFrame()

def fetch_all_stations(api_key: str) -> List[str]:
    """
    Get list of all available station IDs.
    
    Args:
        api_key: Your DMI API key
        
    Returns:
        List of station IDs
    """
    url = "https://dmigw.govcloud.dk/v2/metObs/collections/station/items"
    headers = {"X-API-Key": api_key}
    
    params = {"limit": "1000"}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        stationIds = [feature['properties']['stationIds'] for feature in data['features']]
        return stationIds
    else:
        raise Exception(f"Error fetching stations: {response.status_code} - {response.text}")

def fetch_all_parameters(api_key: str) -> List[str]:
    """
    Get list of all available parameter IDs.
    
    Args:
        api_key: Your DMI API key
        
    Returns:
        List of parameter IDs
    """
    url = "https://dmigw.govcloud.dk/v2/metObs/collections/parameter/items"
    headers = {"X-API-Key": api_key}
    
    params = {"limit": "1000"}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        parameterIds = [feature['properties']['parameterId'] for feature in data['features']]
        return parameterIds
    else:
        raise Exception(f"Error fetching parameters: {response.status_code} - {response.text}")

def filter_active_stations(api_key: str, days_threshold: int = 7) -> List[str]:
    """
    Get only recently active stations.
    
    Args:
        api_key: Your DMI API key
        days_threshold: Number of days to consider a station active
        
    Returns:
        List of active station IDs
    """
    all_stations = fetch_all_stations(api_key)
    
    # Get current time and threshold time
    now = pd.Timestamp.now(tz='UTC')
    threshold = now - pd.Timedelta(days=days_threshold)
    threshold_str = threshold.isoformat()
    
    # Check each station for recent data
    active_stations = []
    headers = {"X-API-Key": api_key}
    
    for station in all_stations:
        params = {
            'stationIds': station,
            'datetime': threshold_str + '/' + now.isoformat(),
            'limit': '1'  # We only need to know if there's at least one record
        }
        
        response = requests.get(BASE_URL, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data.get('features') and len(data['features']) > 0:
                active_stations.append(station)
                print(f"Station {station} is active")
    
    return active_stations


# Example usage
if __name__ == "__main__":
    # Replace with your actual API key
    api_key = "26f63d53-90e3-459c-b7ba-1acb658f2f55"
    
    # Specify the desired start and end time (adjust as needed)
    startTime = pd.Timestamp(2023, 1, 1)
    endTime = pd.Timestamp(2023, 1, 3)  # Short period for testing
    
    # Option 1: Specify stations and parameters manually
    # Specify one or more station IDs or all_stations
    stationIds = [
    '04203', '04208', '04214', '04220', '04228', '04242', '04250',
    '04253', '04266', '04271', '04272', '04285', '04301', '04312',
    '04313', '04320', '04330', '04339', '04351', '04360', '04373',
    '04382', '04390', '05005', '05009', '05015', '05031', '05035',
    '05042', '05065', '05070', '05075', '05081', '05085', '05089',
    '05095', '05105', '05109', '05135', '05140', '05150', '05160',
    '05165', '05169', '05185', '05199', '05202', '05205', '05220',
    '05225', '05269', '05272', '05276', '05277', '05290', '05296',
    '05300', '05305', '05320', '05329', '05343', '05345', '05350',
    '05355', '05365', '05375', '05381', '05395', '05400', '05406',
    '05408', '05435', '05440', '05450', '05455', '05469', '05499',
    '05505', '05510', '05529', '05537', '05545', '05575', '05735',
    '05880', '05889', '05935', '05945', '05970', '05986', '05994',
    '06019', '06031', '06032', '06041', '06049', '06051', '06052',
    '06056', '06058', '06065', '06068', '06072', '06073', '06074',
    '06079', '06081', '06082', '06088', '06093', '06096', '06102',
    '06116', '06119', '06123', '06124', '06126', '06132', '06135',
    '06136', '06138', '06141', '06147', '06149', '06151', '06154',
    '06156', '06159', '06168', '06169', '06174', '06181', '06183',
    '06184', '06186', '06187', '06188', '06193', '06197', '20000',
    '20030', '20055', '20085', '20228', '20279', '20315', '20375',
    '20400', '20552', '20561', '20600', '20670', '21020', '21080',
    '21100', '21120', '21160', '21208', '21368', '21430', '22020',
    '22080', '22162', '22189', '22232', '22410', '23100', '23133',
    '23160', '23327', '23360', '24043', '24102', '24142', '24171',
    '24380', '24430', '24490', '25045', '25161', '25270', '25339',
    '26210', '26340', '26358', '26450', '27008', '27082', '28032',
    '28110', '28240', '28280', '28385', '28552', '28590', '29020',
    '29194', '29243', '29330', '29440', '30075', '30187', '30215',
    '30414', '31040', '31185', '31199', '31259', '31350', '31400',
    '31509', '31570', '32110', '32175', '34270', '34320', '34339']
    parameterIds = [
    # Humdity
    'humidity', 'humidity_past1h',
    # Precipitation
    'precip_past10min', 'precip_past1h', 'precip_past24h',   
    # Temperature
    'temp_dew', 'temp_dry', 'temp_max_past12h', 'temp_max_past1h',
    'temp_mean_past1h', 'temp_min_past12h', 'temp_min_past1h']
    
    # Option 2: Get all stations and parameters (uncomment to use)
    # stationIds = fetch_all_stations(api_key)
    # parameterIds = fetch_all_parameters(api_key)
    # print(f"Found {len(stationIds)} stations and {len(parameterIds)} parameters")
    
    # Option 3: Get only active stations
    # stationIds = filter_active_stations(api_key)
    # parameterIds = ['temp_dry', 'humidity', 'wind_speed', 'precip_past1h']
    # print(f"Found {len(stationIds)} active stations")
    
    # Fetch the data with concurrent requests
    print(f"Fetching data for {len(stationIds)} stations and {len(parameterIds)} parameters...")
    weather_data = get_all_data_concurrent(
        api_key=api_key,
        stationIds=stationIds,
        parameterIds=parameterIds,
        startTime=startTime,
        endTime=endTime,
        max_workers=5  # Adjust based on your system and API rate limits
    )
    
    # Display summary
    print(f"Retrieved data shape: {weather_data.shape}")
    print(weather_data.head())
    
    # Save to CSV
    if not weather_data.empty:
        weather_data.to_csv("dmi_weather_data_concurrent.csv")
        print("Data saved to dmi_weather_data_concurrent.csv")
    else:
        print("No data retrieved.")

Fetching data for 217 stations and 12 parameters...
No data or error for station 04203, parameter precip_past24h
No data or error for station 04203, parameter precip_past1h
No data or error for station 04203, parameter humidity
No data or error for station 04203, parameter precip_past10min
No data or error for station 04203, parameter humidity_past1h
No data for 1/2604: Station 04203, Parameter ['humidity', 'humidity_past1h', 'precip_past10min', 'precip_past1h', 'precip_past24h', 'temp_dew', 'temp_dry', 'temp_max_past12h', 'temp_max_past1h', 'temp_mean_past1h', 'temp_min_past12h', 'temp_min_past1h']
No data for 2/2604: Station 04203, Parameter ['humidity', 'humidity_past1h', 'precip_past10min', 'precip_past1h', 'precip_past24h', 'temp_dew', 'temp_dry', 'temp_max_past12h', 'temp_max_past1h', 'temp_mean_past1h', 'temp_min_past12h', 'temp_min_past1h']
No data for 3/2604: Station 04203, Parameter ['humidity', 'humidity_past1h', 'precip_past10min', 'precip_past1h', 'precip_past24h', 'temp_d

In [10]:
import requests
import json
import pandas as pd
import datetime
import concurrent.futures
from typing import Dict, List, Optional, Union, Tuple

# Base URL for DMI Meteorological Observation API
BASE_URL = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"

def fetch_single_request(api_key: str, 
                        stationId: str, 
                        parameterIds: str, 
                        datetime_str: str,
                        limit: int = 300000) -> Tuple[str, str, pd.DataFrame]:
    """
    Fetch data for a single station-parameter combination.
    """
    headers = {"X-API-Key": api_key}
    params = {
        'datetime': datetime_str,
        'stationIds': stationId,
        'parameterId': parameterIds,
        'limit': str(limit),
    }
    
    try:
        response = requests.get(BASE_URL, headers=headers, params=params)
        if response.status_code == 200:
            json_data = response.json()
            df = pd.json_normalize(json_data.get('features', []))
            if not df.empty:
                df['time'] = pd.to_datetime(df['properties.observed'])
                df = df[['time', 'properties.value', 'properties.stationIds', 'properties.parameterId']]
                df.columns = [c.replace('properties.', '') for c in df.columns]
                df = df[~df.duplicated()]
                return stationId, parameterIds, df
        
        print(f"No data or error for station {stationId}, parameter {parameterIds}")
        return stationId, parameterIds, pd.DataFrame()
        
    except Exception as e:
        print(f"Error for station {stationId}, parameter {parameterIds}: {str(e)}")
        return stationId, parameterIds, pd.DataFrame()

def get_all_data_concurrent(api_key: str,
                           stationIds: List[str],
                           parameterIds: List[str],
                           startTime: pd.Timestamp,
                           endTime: pd.Timestamp,
                           max_workers: int = 10,
                           limit: int = 300000) -> pd.DataFrame:
    """
    Fetch meteorological data for multiple stations and parameters concurrently.
    """
    datetime_str = startTime.tz_localize('UTC').isoformat() + '/' + endTime.tz_localize('UTC').isoformat()
    all_dfs = []
    total_tasks = len(stationIds) * len(parameterIds)
    completed = 0
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for station in stationIds:
            for parameter in parameterIds:
                futures.append(
                    executor.submit(
                        fetch_single_request, 
                        api_key, 
                        station, 
                        parameter, 
                        datetime_str, 
                        limit
                    )
                )
        
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            try:
                stationId, parameterId, df = future.result()
                if not df.empty:
                    print(f"Completed {completed}/{total_tasks}: Station {stationId}, Parameter {parameterId}")
                    all_dfs.append(df)
                else:
                    print(f"No data for {completed}/{total_tasks}: Station {stationId}, Parameter {parameterId}")
            except Exception as e:
                print(f"Error processing task {completed}/{total_tasks}: {str(e)}")
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        combined_df['value'] = pd.to_numeric(combined_df['value'], errors='coerce')
        pivoted_df = combined_df.pivot_table(
            index='time', 
            columns=['parameterId', 'stationIds'], 
            values='value',
            aggfunc='mean'
        )
        return pivoted_df
    else:
        return pd.DataFrame()

def fetch_all_stations(api_key: str) -> List[str]:
    url = "https://dmigw.govcloud.dk/v2/metObs/collections/station/items"
    headers = {"X-API-Key": api_key}
    params = {"limit": "1000"}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        stationIds = [feature['properties']['stationIds'] for feature in data['features']]
        return stationIds
    else:
        raise Exception(f"Error fetching stations: {response.status_code} - {response.text}")

def fetch_all_parameters(api_key: str) -> List[str]:
    url = "https://dmigw.govcloud.dk/v2/metObs/collections/parameter/items"
    headers = {"X-API-Key": api_key}
    params = {"limit": "1000"}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        parameterIds = [feature['properties']['parameterId'] for feature in data['features']]
        return parameterIds
    else:
        raise Exception(f"Error fetching parameters: {response.status_code} - {response.text}")

def filter_active_stations(api_key: str, days_threshold: int = 7) -> List[str]:
    all_stations = fetch_all_stations(api_key)
    now = pd.Timestamp.now(tz='UTC')
    threshold = now - pd.Timedelta(days=days_threshold)
    threshold_str = threshold.isoformat()
    
    active_stations = []
    headers = {"X-API-Key": api_key}
    
    for station in all_stations:
        params = {
            'stationIds': station,
            'datetime': threshold_str + '/' + now.isoformat(),
            'limit': '1'
        }
        response = requests.get(BASE_URL, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get('features') and len(data['features']) > 0:
                active_stations.append(station)
                print(f"Station {station} is active")
    
    return active_stations

# Example usage
if __name__ == "__main__":
    api_key = "26f63d53-90e3-459c-b7ba-1acb658f2f55"
    startTime = pd.Timestamp(2023, 1, 1)
    endTime = pd.Timestamp(2023, 1, 3)

    stationIds = ['04203', '04208']  # Keep short for testing
    parameterIds = ['temp_dry', 'humidity', 'precip_past1h']

    print(f"Fetching data for {len(stationIds)} stations and {len(parameterIds)} parameters...")
    weather_data = get_all_data_concurrent(
        api_key=api_key,
        stationIds=stationIds,
        parameterIds=parameterIds,
        startTime=startTime,
        endTime=endTime,
        max_workers=5
    )

    print(f"Retrieved data shape: {weather_data.shape}")
    print(weather_data.head())

    if not weather_data.empty:
        weather_data.to_csv("dmi_weather_data_concurrent.csv")
        print("Data saved to dmi_weather_data_concurrent.csv")
    else:
        print("No data retrieved.")


Fetching data for 2 stations and 3 parameters...
No data or error for station 04203, parameter precip_past1h
No data for 1/6: Station 04203, Parameter precip_past1h
No data or error for station 04203, parameter temp_dry
No data for 2/6: Station 04203, Parameter temp_dry
No data or error for station 04208, parameter humidity
No data for 3/6: Station 04208, Parameter humidity
No data or error for station 04208, parameter temp_dry
No data or error for station 04203, parameter humidity
No data for 4/6: Station 04208, Parameter temp_dry
No data for 5/6: Station 04203, Parameter humidity
No data or error for station 04208, parameter precip_past1h
No data for 6/6: Station 04208, Parameter precip_past1h
Retrieved data shape: (0, 0)
Empty DataFrame
Columns: []
Index: []
No data retrieved.


In [11]:
import requests
import json
import pandas as pd
import datetime
import concurrent.futures
from typing import Dict, List, Optional, Union, Tuple

# Base URL for DMI Meteorological Observation API
BASE_URL = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"

def fetch_single_request(api_key: str, 
                        stationId: str, 
                        parameterIds: str, 
                        datetime_str: str,
                        limit: int = 300000) -> Tuple[str, str, pd.DataFrame]:
    """
    Fetch data for a single station-parameter combination.
    """
    headers = {"X-API-Key": api_key}
    params = {
        'datetime': datetime_str,
        'stationIds': stationId,
        'parameterId': parameterIds,
        'limit': str(limit),
    }
    
    try:
        response = requests.get(BASE_URL, headers=headers, params=params)
        if response.status_code == 200:
            json_data = response.json()
            df = pd.json_normalize(json_data.get('features', []))
            if not df.empty:
                df['time'] = pd.to_datetime(df['properties.observed'])
                df = df[['time', 'properties.value', 'properties.stationIds', 'properties.parameterId']]
                df.columns = [c.replace('properties.', '') for c in df.columns]
                df = df[~df.duplicated()]
                return stationId, parameterIds, df
        
        print(f"No data or error for station {stationId}, parameter {parameterIds}")
        return stationId, parameterIds, pd.DataFrame()
        
    except Exception as e:
        print(f"Error for station {stationId}, parameter {parameterIds}: {str(e)}")
        return stationId, parameterIds, pd.DataFrame()

def get_all_data_concurrent(api_key: str,
                           stationIds: List[str],
                           parameterIds: List[str],
                           startTime: pd.Timestamp,
                           endTime: pd.Timestamp,
                           max_workers: int = 10,
                           limit: int = 300000) -> pd.DataFrame:
    """
    Fetch meteorological data for multiple stations and parameters concurrently.
    """
    datetime_str = startTime.tz_localize('UTC').isoformat() + '/' + endTime.tz_localize('UTC').isoformat()
    all_dfs = []
    total_tasks = len(stationIds) * len(parameterIds)
    completed = 0
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for station in stationIds:
            for parameter in parameterIds:
                futures.append(
                    executor.submit(
                        fetch_single_request, 
                        api_key, 
                        station, 
                        parameter, 
                        datetime_str, 
                        limit
                    )
                )
        
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            try:
                stationId, parameterId, df = future.result()
                if not df.empty:
                    print(f"Completed {completed}/{total_tasks}: Station {stationId}, Parameter {parameterId}")
                    all_dfs.append(df)
                else:
                    print(f"No data for {completed}/{total_tasks}: Station {stationId}, Parameter {parameterId}")
            except Exception as e:
                print(f"Error processing task {completed}/{total_tasks}: {str(e)}")
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        combined_df['value'] = pd.to_numeric(combined_df['value'], errors='coerce')
        pivoted_df = combined_df.pivot_table(
            index='time', 
            columns=['parameterId', 'stationIds'], 
            values='value',
            aggfunc='mean'
        )
        return pivoted_df
    else:
        return pd.DataFrame()

def fetch_all_stations(api_key: str) -> List[str]:
    url = "https://dmigw.govcloud.dk/v2/metObs/collections/station/items"
    headers = {"X-API-Key": api_key}
    params = {"limit": "1000"}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        stationIds = [feature['properties']['stationIds'] for feature in data['features']]
        return stationIds
    else:
        raise Exception(f"Error fetching stations: {response.status_code} - {response.text}")

def fetch_all_parameters(api_key: str) -> List[str]:
    url = "https://dmigw.govcloud.dk/v2/metObs/collections/parameter/items"
    headers = {"X-API-Key": api_key}
    params = {"limit": "1000"}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        parameterIds = [feature['properties']['parameterId'] for feature in data['features']]
        return parameterIds
    else:
        raise Exception(f"Error fetching parameters: {response.status_code} - {response.text}")

def filter_active_stations(api_key: str, days_threshold: int = 7) -> List[str]:
    all_stations = fetch_all_stations(api_key)
    now = pd.Timestamp.now(tz='UTC')
    threshold = now - pd.Timedelta(days=days_threshold)
    threshold_str = threshold.isoformat()
    
    active_stations = []
    headers = {"X-API-Key": api_key}
    
    for station in all_stations:
        params = {
            'stationIds': station,
            'datetime': threshold_str + '/' + now.isoformat(),
            'limit': '1'
        }
        response = requests.get(BASE_URL, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get('features') and len(data['features']) > 0:
                active_stations.append(station)
                print(f"Station {station} is active")
    
    return active_stations

# Example usage
if __name__ == "__main__":
    api_key = "26f63d53-90e3-459c-b7ba-1acb658f2f55"
    startTime = pd.Timestamp(2023, 1, 1)
    endTime = pd.Timestamp(2023, 1, 3)

    stationIds = ['04203', '04208']  # Keep short for testing
    parameterIds = ['temp_dry', 'humidity', 'precip_past1h']

    print(f"Fetching data for {len(stationIds)} stations and {len(parameterIds)} parameters...")
    weather_data = get_all_data_concurrent(
        api_key=api_key,
        stationIds=stationIds,
        parameterIds=parameterIds,
        startTime=startTime,
        endTime=endTime,
        max_workers=5
    )

    print(f"Retrieved data shape: {weather_data.shape}")
    print(weather_data.head())

    if not weather_data.empty:
        weather_data.to_csv("dmi_weather_data_concurrent.csv")
        print("Data saved to dmi_weather_data_concurrent.csv")
    else:
        print("No data retrieved.")


Fetching data for 2 stations and 3 parameters...
No data or error for station 04203, parameter temp_dry
No data or error for station 04203, parameter humidity
No data or error for station 04208, parameter humidity
No data or error for station 04203, parameter precip_past1h
No data or error for station 04208, parameter temp_dry
No data for 1/6: Station 04203, Parameter temp_dry
No data for 2/6: Station 04203, Parameter humidity
No data for 3/6: Station 04208, Parameter humidity
No data for 4/6: Station 04203, Parameter precip_past1h
No data for 5/6: Station 04208, Parameter temp_dry
No data or error for station 04208, parameter precip_past1h
No data for 6/6: Station 04208, Parameter precip_past1h
Retrieved data shape: (0, 0)
Empty DataFrame
Columns: []
Index: []
No data retrieved.
